In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Подключите необходимые библиотеки.**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(12, 10)});

**Загрузите данные. Они находятся в директории input.**

In [ ]:
df = pd.read_csv('../input/bigml_59c28831336c6604c800002a.csv')
df.info()

**Каково распределение абонентов по целевому признаку *churn*? Назовите процент лояльных абонентов.**

In [ ]:
new_values = {False: 'loyal', True: 'disloyal'}
df['named_churn'] = df['churn'].map(new_values)
by_churn = df['named_churn'].value_counts()
by_churn.plot(kind='bar')
print(f"Percentage of loyal accounts: {by_churn['loyal'] / by_churn.sum()  * 100}%\n")

Жители какого штата говорят по телефону больше всего? меньше всего? Обратите внимание, что нас интересуют все виды звонков.

In [ ]:
df['all calls'] = df['total day calls'] + df['total eve calls'] + df['total night calls']
state_calls = df.groupby('state')['all calls'].sum().sort_values()
print('State with the minimum amount of calls:\n', state_calls[state_calls == state_calls[0]], sep='')
print('State with the maximum amount of calls:\n', state_calls[state_calls == state_calls[-1]], sep='')
plt.xlabel('Amount of calls')
plt.ylabel('State')
state_calls.plot(kind='barh', x='Amount of calls', y='State', figsize=(20, 15), colormap='ocean')

Как видим, наименьшее количество звонков сделано из Калифорнии, а наибольшее - из Западной Вирджинии.

**Верно ли, что люди, совершающие много международных звонков, приобретают *international plan*? Не забывайте о визуализации информации.**

In [ ]:
sns.countplot(y='total intl calls', hue='international plan', data=df)

Данное предположение неверно.

**Как связаны наличие голосовой почты и международного плана с лояльностью абонентов? Проверьте статистическую значимость этих связей.**

In [ ]:
sns.heatmap(pd.crosstab(df['international plan'], df['churn']), 
            cmap="YlGnBu", annot=True, cbar=False);

In [ ]:
sns.heatmap(pd.crosstab(df['voice mail plan'], df['churn']), 
            cmap="YlGnBu", annot=True, cbar=False);

In [ ]:
from scipy.stats import chi2_contingency
print('International plan vs Churn:')
chi2_contingency(pd.crosstab(df['international plan'], df['churn']))

In [ ]:
print('Voice mail plan vs Churn:')
chi2_contingency(pd.crosstab(df['voice mail plan'], df['churn']))

p-value в обоих случаях доволььно мало (<0.05). Значит, связи международного плана и голосовой почты с лояльностью абонента статистически значима.

**Что можно сказать о взаимосвязи количества обращений в колл-центр и лояльности? Не забывайте о визуализации информации.**

In [ ]:
sns.countplot(y='customer service calls', hue='churn', data=df)

In [ ]:
from scipy.stats import pointbiserialr
pointbiserialr(df['churn'], df['customer service calls'])

Как можно увидеть, связь между лояльностью и количеством звонков в колл-центр подтвердилась.

**Какие из числовых признаков имеют наибольшую степень корреляции? Почему?**

In [ ]:
numeric = [col for col in df.columns if df.dtypes[col] == np.int64 or df.dtypes[col] == np.float64]
df[numeric].corr(method='spearman')

In [ ]:
sns.heatmap(df[numeric].corr(method='spearman'));

Наибольшая корреляция прослеживается между общим количеством минут разговоров и общим платежом за услуги. Очевидно, что при увеличении количества разговорных минут увеличивается и оплата, т.к. она зависит напрямую от количества минут.

**Связано ли время пользования аккаунтом с количеством обращений в колл-центр?**

In [ ]:
from scipy.stats import spearmanr, kendalltau
r = spearmanr(df['account length'], df['customer service calls'])
t = kendalltau(df['account length'], df['customer service calls'])
print('Spearman correlation between account length and customer service calls:', r[0], 'p-value:', r[1])
print('Kendall correlation between account length and customer service calls:', t[0], 'p-value:', t[1])
plt.xlabel('account length')
plt.ylabel('customer service calls')
plt.scatter(df['account length'], df['customer service calls'])

Между признаками возможна обратная слабая корреляция, но значение p-value достаточно велико, следовательно, есть большая вероятность наличия статистической ошибки.

**Какие ещё инсайты можно найти в имеющихся данных? Творческое задание.**

**Проверим связь количества отправленных голосовых писем с лояльностью абонентов.**

In [ ]:
sns.countplot(y='number vmail messages', hue='churn', data=df[df['voice mail plan'] == 'yes'])

In [ ]:
from scipy.stats import pointbiserialr
pointbiserialr(df['churn'], df['number vmail messages'])

Прослеживается довольно слабая обратная корреляция.

**Посмотрим распределение лояльных и нелояльных абонентов по штатам и выделим те, где больше всего и меньше всего лояльных абонентов.**

In [ ]:
sns.countplot(y='state', hue='churn', data=df)

In [ ]:
loyal_counts = df[df['churn'] == False]['state'].value_counts()
disloyal_counts = df[df['churn'] == True]['state'].value_counts()
loyalty_distribution = round(loyal_counts / (loyal_counts + disloyal_counts) * 100, 2)
print('A state with the highest percentage of loyalty:', loyalty_distribution.idxmax())
print('A state with the lowest percentage of loyalty:', loyalty_distribution.idxmin())
loyalty_distribution.sort_values().plot(kind='barh', colormap='ocean')

Можно сделать вывод, что наиболее лояльным штатом являются Гавайи и Аляска, а наиболее нелояльным - Калифорния и Нью-Джерси.

**Проверим, существует ли связь между общей платой за услуги с лояльностью абонентов.**

In [ ]:
df['full charge'] = df['total day charge'] + df['total eve charge'] + df['total night charge']
pointbiserialr(df['churn'], df['full charge'])

Можем увидеть, что существует корреляция, причем связь между этими признаками статистически подтверждена.

In [ ]:
df.groupby('churn')['full charge'].mean().plot(kind='bar') 
plt.ylabel('Full charge')
plt.show();

Можно сказать, что нелояльные абоненты в среднем больше тратят средств на звонки, чем лояльные. На следующем графике при выборе медианы в наборе платежей наблюдается то же самое.

In [ ]:
df.groupby('churn')['full charge'].median().plot(kind='bar') 
plt.ylabel('Full charge')
plt.show();